In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import scanpy as sc
from muon import MuData
import muon as mu

### Read Data

In [ ]:
DataPathway = ""

In [ ]:
filePathway = "%sshare.h5mu" %DataPathway
mdata = mu.read(filePathway)

In [ ]:
mdata

### Split Datasets

##### Train : Validation : Test = 70% : 10% : 20%

In [ ]:
Train_Validation_barcode, Test_barcode =  train_test_split(mdata.obs_names, test_size =.20, shuffle  = True) 
Train_barcode, Validation_barcode =  train_test_split(Train_Validation_barcode, test_size =.125, shuffle  = True) 
len(Train_barcode), len(Validation_barcode), len(Test_barcode)

In [ ]:
Train = mdata[Train_barcode]
Validation = mdata[Validation_barcode]
Test = mdata[Test_barcode]

## Train

### PCA for scRNA-seq and Get the 30 Neighbors

In [ ]:
sc.tl.pca(Train.mod['RNA'])
sc.pp.neighbors(Train.mod['RNA'], n_neighbors=31, n_pcs=30)
Train.mod['RNA']

In [ ]:
Distance_index = Train.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Train_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Train_knn_neighbor.index = [Train.mod['RNA'].obs_names]
Train_knn_neighbor

### Get 30 neighbors expression values of the cell

In [ ]:
sc.get.var_df(Train.mod['RNA'], keys=Train.mod['RNA'].obs_names[Train_knn_neighbor.iloc[6]].tolist())

## Test

In [ ]:
sc.tl.pca(Test.mod['RNA'])
sc.pp.neighbors(Test.mod['RNA'], n_neighbors=31, n_pcs=30)
Test.mod['RNA']

In [ ]:
Distance_index = Test.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Test_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Test_knn_neighbor.index = [Test.mod['RNA'].obs_names]

## Validation

In [ ]:
sc.tl.pca(Validation.mod['RNA'])
sc.pp.neighbors(Validation.mod['RNA'], n_neighbors=31, n_pcs=30)
Validation.mod['RNA']

In [ ]:
Distance_index = Validation.mod['RNA'].obsp['distances'].toarray().argsort(axis=1)[:, -30:]
col_names = ["Neigh_{}".format(i) for i in range(1, 31)]
Validation_knn_neighbor = pd.DataFrame(Distance_index, columns=col_names)
Validation_knn_neighbor.index = [Validation.mod['RNA'].obs_names]

### Save object

In [ ]:
Train.write("%sdatasets/Train.h5mu"%DataPathway)
Test.write("%sdatasets/Test.h5mu"%DataPathway)
Validation.write("%sdatasets/Validation.h5mu"%DataPathway)

### Save Neighbor Table

In [ ]:
Train_knn_neighbor.to_csv("%sdatasets/Train_neighbor.csv"%DataPathway, index=True)
Test_knn_neighbor.to_csv("%sdatasets/Test_neighbor.csv"%DataPathway, index=True)
Validation_knn_neighbor.to_csv("%sdatasets/Validation_neighbor.csv"%DataPathway, index=True)